## 6.5 Memory using SQLite

<div style="text-align: right"> Initial issue : 2025.06.16 </div>
<div style="text-align: right"> last update : 2025.06.16 </div>

대화이력을 서비스 종료 이후에도 기억해두었다가 불러올 때 활용

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import SQLChatMessageHistory

### 사용방법

storage를 사용하려면 다음 2가지 설정 필요

1. `session_id` - 사용자 이름, 이메일, 채팅 ID 등과 같은 세션의 고유 식별자.

2. `connection` - 데이터베이스 연결을 지정하는 문자열. 이 문자열은 SQLAlchemy의 create_engine 함수에 전달됨.

In [3]:
# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 파일을 설정
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history", connection="sqlite:///sqlite.db"
)

In [4]:
# 사용자 메시지를 추가
chat_message_history.add_user_message(
    "안녕? 만나서 반가워. 내 이름은 홍길동이야"
)
# AI 메시지를 추가
chat_message_history.add_ai_message("안녕 홍길동, 만나서 반가워. 나도 잘 부탁해!")

저장된 메시지 확인 - `chat_message_history.messages`

In [5]:
# 채팅 메시지 기록의 메시지들
chat_message_history.messages

[HumanMessage(content='안녕? 만나서 반가워. 내 이름은 홍길동이야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 홍길동, 만나서 반가워. 나도 잘 부탁해!', additional_kwargs={}, response_metadata={})]

### LCEL에 적용하기

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        # 시스템 메시지
        ("system", "You are a helpful assistant."),
        # 대화 기록을 위한 Placeholder
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),  # 질문
    ]
)

In [7]:
# chain 생성
chain = prompt | ChatOpenAI(model_name="gpt-4o") | StrOutputParser()

`sqlite.db` 에서 대화내용을 가져오는 함수를 정의

In [8]:

def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id, # 사용자 마다 여러개의 대화이력을 가질 수 있으므로 구분분
        connection="sqlite:///sqlite.db",
    )

Config field 정의하기 - user_id, conversation_id 필드를 정의

In [9]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="user_id", # get_chat_history 함수의 인자와 동일해야함
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id", # get_chat_history 함수의 인자와 동일해야함
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

history를 포함하는 chain 정의

In [10]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,  # 대화 기록을 가져오는 함수를 설정
    input_messages_key="question",  # 입력 메시지의 키를 "question"으로 설정
    history_messages_key="chat_history",  # 대화 기록 메시지의 키를 "history"로 설정
    history_factory_config=config_fields,  # 대화 기록 조회시 참고할 파라미터를 설정
)

(중요) `"configurable"` 키 아래에 `"user_id"`, `"conversation_id"` key-value 쌍을 설정

In [11]:
# config 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

질문 진행하기

- `chain_with_history` 객체의 `invoke` 메서드를 호출하여 질문에 대한 답변을 생성
- `invoke` 메서드에는 질문 딕셔너리와 `config` 설정이 전달

In [12]:
# 질문과 config 를 전달하여 실행합니다.
chain_with_history.invoke({"question": "안녕 반가워, 내 이름은 홍길동이야"}, config)

'안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [13]:
# 후속 질문을 실행
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)

'당신의 이름은 홍길동이라고 소개하셨어요. 맞나요?'

같은 `user_id` 를 가지지만 `conversion_id` 가 다른 값을 가지도록 설정하고 질문

In [14]:
# config 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation2"}}

# 질문과 config 를 전달하여 실행
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)

'죄송하지만, 저는 사용자님의 이름을 알 수 없습니다. 당신이 원하는 이름을 말씀해주시면 그 이름으로 호칭해드리겠습니다.'